In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split

# CSV'yi yükle
df = pd.read_csv("/content/drive/Shareddrives/GDG - AI/News-Title-Sum/dataset_clean.csv")

# Train/Val/Test ayır
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Hugging Face Dataset formatına dönüştür
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
val_dataset = Dataset.from_pandas(val_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/mt5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    texts = [str(t) for t in examples["text"]]
    titles = [str(t) for t in examples["title"]]

    model_inputs = tokenizer(
        texts,
        max_length=512,
        padding="max_length",
        truncation=True
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            titles,
            max_length=32,
            padding="max_length",
            truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)



Map:   0%|          | 0/8154 [00:00<?, ? examples/s]

Map:   0%|          | 0/1019 [00:00<?, ? examples/s]

In [ ]:
train_dataset = train_dataset.filter(lambda x: x["text"] is not None and x["title"] is not None)
val_dataset = val_dataset.filter(lambda x: x["text"] is not None and x["title"] is not None)


In [ ]:
!pip install rouge_score --quiet


  Preparing metadata (setup.py) ... done


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

import os
os.environ["WANDB_DISABLED"] = "true"

from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/Shareddrives/GDG - AI/News-Title-Sum/title_model",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    num_train_epochs=10,
    predict_with_generate=True,
    logging_dir="/content/drive/Shareddrives/GDG - AI/News-Title-Sum/title_logs",

    # ✅ Epoch sonunda checkpoint alınsın
    save_strategy="epoch",

    # ✅ Her epoch başında evaluation yapılsın
    eval_steps=len(tokenized_train),  # 1 epoch = tüm örnekler

    # ✅ Sadece son 3 checkpoint kalsın
    save_total_limit=3,

    # ✅ WANDB kapalı
    report_to="none",

    # ✅ Loglar sade ve epoch bazlı
    logging_steps=len(tokenized_train)  # Epoch sonunda log gösterimi
)

from evaluate import load
import numpy as np

rouge = load("rouge")

def compute_metrics(eval_preds):
    predictions, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # ROUGE hesapla
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # Eval loss zaten Trainer tarafından loglanıyor, biz sadece metrikleri döndürüyoruz
    return {
        "rouge1": result["rouge1"],
        "rouge2": result["rouge2"],
        "rougeL": result["rougeL"],
    }




trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)



trainer.train()



<ipython-input-15-e76af6cb0399>:58: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss
8154,1.195500


TrainOutput(global_step=10200, training_loss=1.1274486825980392, metrics={'train_runtime': 3976.2609, 'train_samples_per_second': 20.507, 'train_steps_per_second': 2.565, 'total_flos': 9.777022157979648e+16, 'train_loss': 1.1274486825980392, 'epoch': 10.0})

In [ ]:
save_path = "/content/drive/Shareddrives/GDG - AI/News-Title-Sum/title_model/final"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model ve tokenizer başarıyla {save_path} klasörüne kaydedildi.")


Model ve tokenizer başarıyla /content/drive/Shareddrives/GDG - AI/News-Title-Sum/title_model/final klasörüne kaydedildi.


In [ ]:
import pandas as pd

# Test dataframe'i al (önceden ayırmıştık)
test_df = test_df.reset_index(drop=True)

# Tahminler için boş liste
predictions = []

# Test örnekleri üzerinde döngü
for text in test_df["text"]:
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=512
    ).to(model.device)

    output_ids = model.generate(
        inputs["input_ids"],
        max_length=32,
        num_beams=4,
        early_stopping=True
    )

    pred = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    predictions.append(pred)

# Tahminleri test_df'e ekle
test_df["predicted_title"] = predictions


In [ ]:
from evaluate import load

rouge = load("rouge")

# Gerçek başlıklar
references = test_df["title"].tolist()

# ROUGE hesapla
results = rouge.compute(predictions=predictions, references=references, use_stemmer=True)

# Sonuçları yazdır
print("🔍 ROUGE Skorları:")
print(f"ROUGE-1: {results['rouge1']:.4f}")
print(f"ROUGE-2: {results['rouge2']:.4f}")
print(f"ROUGE-L: {results['rougeL']:.4f}")

🔍 ROUGE Skorları:
ROUGE-1: 0.3738
ROUGE-2: 0.2389
ROUGE-L: 0.3576


In [ ]:
output_csv_path = "/content/drive/Shareddrives/GDG - AI/News-Title-Sum/test_predictions.csv"
test_df.to_csv(output_csv_path, index=False, encoding="utf-8")
print(f"Tahminler {output_csv_path} dosyasına kaydedildi.")

Tahminler /content/drive/Shareddrives/GDG - AI/News-Title-Sum/test_predictions.csv dosyasına kaydedildi.


In [ ]:
import random

# CSV dosyasını yükle
df = pd.read_csv("/content/drive/Shareddrives/GDG - AI/News-Title-Sum/test_predictions.csv")

# Rastgele 5 örnek seç
samples = df.sample(n=5, random_state=42)

# Her biri için metin, gerçek başlık ve tahmin edilen başlığı yazdır
for i, row in samples.iterrows():
    print(f"\n📰 Haber Metni:\n{row['text']}\n")
    print(f"✅ Gerçek Başlık: {row['title']}")
    print(f"🔮 Tahmin Edilen Başlık: {row['predicted_title']}")
    print("-" * 100)



📰 Haber Metni:
Suriye'de 61 yıllık Baas rejiminin çökmesiyle eş zamanlı olarak İsrail ordusunun ülkeye saldırıları ve işgali arttı. İsrail ordusu Suriye'de işgal ettiği bölgelere askeri noktalar inşa ediyor ve oradan Suriye'deki köyleri izliyor. İşte A Haber muhabiri Mehmet Geçgel ve Kameraman Mehmet Ali Bağ'ın özel görüntüleri… Suriye'de 61 yıllık Baas rejiminin çökmesiyle eş zamanlı olarak İsrail ordusunun ülkeye saldırıları ve işgali arttı. İsrail ordusu Suriye'de işgal ettiği bölgelere askeri noktalar inşa ediyor ve oradan Suriye'deki köyleri izliyor. İşte A Haber muhabiri Mehmet Geçgel ve Kameraman Mehmet Ali Bağ'ın özel görüntüleri… Soykırımcı İsrail ordusu, Suriye'de işgal ettiği bölgelere askeri noktalar kurarak çevredeki köyleri gözetliyor. İsrail'in işgal ettiği köylerden birine giren Mehmet Geçgel bölgeden yaşananları aktardı. İŞGAL YERLERİNE ASKERİ ÜS Mehmet Geçgel: Kuneytra bölgesinde İsrail'in işgal ettiği köye girdik. Arka taraftan dolaşarak girdik. Çünkü hemen ön taraf

In [ ]:
def predict_title(txt):
    # Metni modele uygun şekilde tokenize et
    inputs = tokenizer(
        txt,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=512
    ).to(model.device)

    # Modelden tahmin al
    output_ids = model.generate(
        inputs["input_ids"],
        max_length=32,
        num_beams=4,
        early_stopping=True
    )

    # Tahmini decode et
    predicted_title = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return predicted_title
txt = """
Kanser tedavisi gören 65 yaşındaki oyuncu ve şair Yavuz Çetin, hayatını kaybetti. Sanatçı, bugün Amasya'da düzenlenecek cenaze töreniyle son yolculuğuna uğurlanacak.
Oyuncu ve şair kimliğiyle tanınan Yavuz Çetin, bir süredir mücadele ettiği kanser hastalığına yenik düşerek 65 yaşında hayatını kaybetti.

Acı haberi ailesi sosyal medya hesaplarından duyurdu. Çetin'in vefat haberi sanat dünyasında ve sevenleri arasında derin bir üzüntüye yol açtı. Evli ve iki çocuk babası olan Yavuz Çetin'in cenazesi, bugün öğle namazının ardından Sultan II. Bayezid Camii'nde kılınacak cenaze namazıyla uğurlanacak. Merhum sanatçı, Tekirdede Mezarlığı'nda toprağa verilecek.
"""

tahmin = predict_title(txt)
print("🔮 Tahmin Edilen Başlık:", tahmin)


🔮 Tahmin Edilen Başlık: Oyuncu ve şair Yavuz Çetin hayatını kaybetti
